## Yo this notebook is lit

In [19]:
import pandas as pd
import numpy as np
from skimage.io import imshow
from sklearn.model_selection import train_test_split

In [63]:
import keras
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
y_train_labels = [class_labels[int(val)] for val in y_train]
h = 32
w = 32

X_train = X_train/255.0
X_test = X_test/255.0
X_train = X_train - 1/2
X_test = X_test - 1/2

# Takes only the classes we are concerned with 
sub_sample = [1, 4, 9]
X_sub_train = []
y_sub_train = []
for i in range(0,len(y_train)):
    if y_train[i][0] in sub_sample:
        X_sub_train.append(X_train[i])
        if y_train[i][0] == 1 or y_train[i][0] == 9:
            y_sub_train.append([0])
        if y_train[i][0] == 4:
            y_sub_train.append([1])

        
X_sub_test = []
y_sub_test = []
for i in range(0,len(y_test)):
    if y_test[i][0] in sub_sample:
        X_sub_test.append(X_test[i])
        # If it's a truck or automobile
        if y_test[i][0] == 1 or y_test[i][0] == 9:
            y_sub_test.append([0])
        # If it's a deer
        if y_test[i][0] == 4:
            y_sub_test.append([1])

X_sub_train = np.array(X_sub_train)
X_sub_test = np.array(X_sub_test)
y_sub_train = np.array(y_sub_train)
y_sub_test = np.array(y_sub_test)
        
sub_class_labels = ['automobile', 'deer']
y_sub_train_labels = [sub_class_labels[int(val)] for val in y_sub_train]

NUM_CLASSES = len(sub_class_labels)
y_sub_train_ohe = keras.utils.to_categorical(y_sub_train, NUM_CLASSES)
y_sub_test_ohe = keras.utils.to_categorical(y_sub_test, NUM_CLASSES)

In [57]:
#print(X_sub_train)
#print(X_train)

In [20]:
imshow(X_sub_train[0].squeeze())

RuntimeError: No suitable plugin registered for imshow.

You may load I/O plugins with the `skimage.io.use_plugin` command.  A list of all available plugins are shown in the `skimage.io` docstring.

In [64]:
print(len(X_sub_train))
print(len(y_sub_train))
print(len(X_sub_test))
print(len(y_sub_test))

15000
15000
3000
3000


In [5]:
#X_c = np.concatenate([X_train,X_test], axis=0)
#y_c = np.concatenate([y_train,y_test], axis=0).flatten()
#vectorized_color = np.reshape(X_c,(len(X_c),32*32,3))
#color_pixels = np.reshape(vectorized_color,(len(vectorized_color)*32*32,3))

In [41]:
#bw_pixels = [(0.21*pixel[0])+(0.72*pixel[1])+(0.07*pixel[2]) for pixel in color_pixels]#convert rgb to b/w
#vectorized_bw = np.reshape(bw_pixels,(len(X),32*32))#reconstruct to list in image vectors
#vectorized_bw.dump("data/vectorized_bw")

In [48]:
'''vectorized_bw_all = np.load("data/vectorized_bw")
print(vectorized_bw_all.shape)

#we might need these DF's later
df_bw = pd.DataFrame(data=vectorized_bw_all)
df_bw = df_bw.assign(y=pd.Series(y,name='y'))
#df_bw_sample = df_bw.sample(5000)
X = df_bw.drop('y', axis=1).as_matrix()
y = df_bw.y.as_matrix()
print(X.shape)
print(y.shape)'''
#now, we can do cv or a train test split

(60000, 1024)
(60000, 1024)
(60000,)


In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

NameError: name 'X' is not defined

In [52]:
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.layers import average 
from keras.models import Input, Model
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split

In [65]:
datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=5, # used, Int. Degree range for random rotations.
    width_shift_range=0.1, # used, Float (fraction of total width). Range for random horizontal shifts.
    height_shift_range=0.1, # used,  Float (fraction of total height). Range for random vertical shifts.
    shear_range=0., # Float. Shear Intensity (Shear angle in counter-clockwise direction as radians)
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None)

datagen.fit(X_sub_train)

In [68]:
%%time

from keras.layers import average 
from keras.models import Input, Model
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 
l2_lambda = 0.0001

num_ensembles = 3

input_holder = Input(shape=(w, h, 3))

branches = []
for _ in range(num_ensembles):

    conv1 = Conv2D(filters=32,
                   input_shape = (w,h,1),
                   kernel_size=(3,3),
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda),
                   padding='same', 
                   activation='relu')(input_holder)
    
    max1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv1)

    conv2 = Conv2D(filters=32,
                   kernel_size=(3,3),
                   kernel_initializer='he_uniform', 
                   kernel_regularizer=l2(l2_lambda),
                   padding='same', 
                   activation='relu')(max1)
    
    max2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv2)


    # add one layer on flattened output
    drop1 = Dropout(0.25)(max2) # add some dropout for regularization after conv layers
    flat1 = Flatten()(drop1)
    dense1 = Dense(128, 
                  activation='relu',
                  kernel_initializer='he_uniform',
                  kernel_regularizer=l2(l2_lambda)
                )(flat1)
    drop2 = Dropout(0.5)(dense1) # add some dropout for regularization, again!
    dense2 = Dense(NUM_CLASSES, 
                  activation='sigmoid', 
                  kernel_initializer='glorot_uniform',
                  kernel_regularizer=l2(l2_lambda)
                 )(drop2)
    
    # now add this branch onto the master list
    branches.append(dense2)

# that's it, we just need to average the results
ave = average(branches)

# here is the secret sauce for setting the network using the 
#   Model API:
cnn_ens = Model(inputs=input_holder,outputs=ave)

# Let's train the model 
cnn_ens.compile(loss='categorical_crossentropy', # 'categorical_crossentropy' 'mean_squared_error'
                optimizer='rmsprop', # 'adadelta' 'rmsprop'
                metrics=['accuracy'])

# the flow method yields batches of images indefinitely, with the given transofmrations
cnn_ens.fit_generator(datagen.flow(X_sub_train, y_sub_train_ohe, batch_size=128), 
                      steps_per_epoch=int(len(X_sub_train)/128), # how many generators to go through per epoch
                      epochs=2, verbose=1,
                      validation_data=(X_sub_test,y_sub_test_ohe),
                      callbacks=[EarlyStopping(monitor='val_loss', patience=4)]
                     )

Epoch 1/2
117/117 [==============================] - 5s - loss: 0.4301 - acc: 0.8502 - val_loss: 0.2810 - val_acc: 0.9160
Epoch 2/2
117/117 [==============================] - 4s - loss: 0.2785 - acc: 0.9211 - val_loss: 0.2254 - val_acc: 0.9403
Wall time: 11.5 s


In [70]:
from sklearn import metrics as mt
yhat_cnn = np.argmax(cnn_ens.predict(X_sub_test), axis=1)
rec_cnn = mt.f1_score(y_sub_test,yhat_cnn, average='macro')
acc_cnn = mt.accuracy_score(y_sub_test,yhat_cnn)
print("Recall: ", rec_cnn)
print("Accuracy: ", acc_cnn)

Recall:  0.930283995997
Accuracy:  0.940333333333


In [19]:
for a in yhat_cnn:
    print(a)

3
8
8
8
4
6
1
6
3
1
8
9
5
7
9
8
5
3
8
6
7
0
0
9
4
4
3
0
9
6
6
2
4
3
9
9
7
9
9
3
0
6
3
6
9
9
3
9
4
6
9
8
2
7
8
8
4
5
3
3
7
3
1
3
6
6
1
8
3
7
0
3
8
8
9
4
9
3
5
8
8
1
1
7
2
7
2
8
8
9
0
4
8
6
4
6
6
0
0
3
4
3
6
3
1
1
2
6
7
5
6
0
2
2
1
7
0
4
3
4
8
2
1
2
8
1
8
3
6
0
4
9
9
9
1
2
9
7
4
8
4
5
6
5
8
3
6
6
5
3
1
1
6
9
0
3
2
9
6
4
0
1
9
6
8
7
8
6
5
0
9
9
1
8
9
9
3
7
3
9
8
5
2
3
3
8
6
3
3
8
6
8
9
1
3
4
8
2
3
8
5
1
8
7
1
3
8
2
7
9
3
1
3
9
8
3
4
9
0
4
7
3
9
5
3
9
3
2
3
6
3
1
5
8
8
0
4
9
3
3
1
1
1
9
0
8
1
8
2
2
5
5
9
1
4
8
6
1
8
1
8
1
9
3
0
8
3
4
7
0
4
2
6
5
8
3
9
7
4
9
9
1
6
1
9
9
1
8
7
9
1
2
6
1
3
4
6
0
0
6
6
6
3
0
2
1
9
2
9
2
8
6
8
0
3
7
7
4
5
3
5
3
4
2
7
9
7
3
9
6
1
9
6
6
6
1
6
8
0
7
4
6
0
4
8
5
7
6
8
9
1
1
8
0
6
5
4
2
1
0
9
5
8
0
9
7
2
9
8
1
4
7
9
4
2
7
8
7
3
6
6
9
0
9
2
8
7
4
5
5
1
8
6
2
1
6
2
3
1
1
9
1
7
8
8
6
0
9
8
3
7
0
5
6
9
9
3
5
2
7
4
7
8
0
2
9
3
4
6
3
4
5
3
7
9
7
9
4
5
1
1
4
9
9
2
4
5
0
2
0
1
9
7
3
9
2
4
4
6
8
5
6
1
4
3
4
7
3
5
8
4
7
8
0
7
7
6
9
5
3
9
6
8
5
6
9
9
9
3
0
1
0
8
1
1
8
8
5
4
0


8
4
7
9
9
2
8
9
1
8
9
9
9
1
7
1
1
1
7
6
3
4
0
0
1
2
3
1
1
1
8
8
8
9
8
0
4
6
3
5
3
6
3
1
2
7
7
8
0
1
4
9
7
6
2
5
2
6
6
6
0
8
1
5
3
3
8
1
1
6
1
9
2
3
8
4
7
4
0
0
7
6
8
9
1
6
7
8
9
2
9
1
0
7
7
4
6
5
3
2
6
6
7
4
5
0
0
9
9
7
2
7
2
3
2
7
6
4
6
4
9
6
8
3
8
5
7
6
3
4
5
9
7
7
1
8
0
7
8
1
9
3
0
8
2
6
0
8
1
4
1
3
5
1
8
9
6
4
6
1
1
3
3
9
3
4
3
9
9
5
4
5
1
3
9
3
6
9
9
9
4
0
5
1
6
3
8
2
5
3
5
5
8
0
5
9
4
6
7
0
8
9
2
3
3
7
4
5
8
8
7
8
5
7
7
4
8
9
7
8
4
8
4
4
2
6
4
8
7
2
7
0
4
3
2
7
1
1
3
9
9
1
0
6
4
5
0
9
0
6
2
0
4
1
3
7
3
1
0
4
6
0
4
5
4
7
6
4
3
5
9
4
5
2
0
6
7
4
3
9
4
5
1
1
8
3
9
8
5
8
6
9
3
4
0
2
3
2
6
1
8
6
9
7
3
2
2
5
3
5
0
4
1
4
8
9
8
3
2
7
1
7
9
8
4
6
1
1
8
6
3
5
8
9
4
7
0
8
4
1
1
8
3
7
5
8
2
9
3
8
9
6
8
9
9
4
8
9
5
5
2
4
5
1
8
1
2
5
1
3
9
6
3
0
8
4
2
8
7
6
7
6
9
6
3
5
1
1
5
7
9
5
8
1
6
0
8
9
4
6
4
6
5
1
1
7
8
6
8
9
9
6
1
0
1
1
7
9
1
3
4
6
0
3
6
3
3
8
9
8
9
3
1
3
7
3
1
4
9
9
8
1
7
2
9
8
1
3
6
0
9
5
0
4
6
3
1
1
0
8
3
6
9
3
4
6
1
6
7
9
6
4
6
5
5
8
3
9
2
0
9
4
0
0
1
4
4
5
5
9
5
6
6
2
3
9
3
5
3
9


In [59]:
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 
l2_lambda = 0.0001
img_wh = 32

# Use Kaiming He to regularize ReLU layers: https://arxiv.org/pdf/1502.01852.pdf
# Use Glorot/Bengio for linear/sigmoid/softmax: http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf 
cnn = Sequential()

cnn.add(Conv2D(filters=32,
               input_shape = (img_wh,img_wh,1),
               kernel_size=(3,3),
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda),
               padding='same', 
               activation='relu')) # more compact syntax
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

cnn.add(Conv2D(filters=32,
               kernel_size=(3,3),
               kernel_initializer='he_uniform', 
               kernel_regularizer=l2(l2_lambda),
               padding='same', 
               activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))
    

# add one layer on flattened output
cnn.add(Dropout(0.25)) # add some dropout for regularization after conv layers
cnn.add(Flatten())
cnn.add(Dense(128, 
              activation='relu',
              kernel_initializer='he_uniform',
              kernel_regularizer=l2(l2_lambda)
       ))
cnn.add(Dropout(0.5)) # add some dropout for regularization, again!
cnn.add(Dense(NUM_CLASSES, 
              activation='softmax', 
              kernel_initializer='glorot_uniform',
              kernel_regularizer=l2(l2_lambda)
             ))

# Let's train the model 
cnn.compile(loss='categorical_crossentropy', # 'categorical_crossentropy' 'mean_squared_error'
              optimizer='rmsprop', # 'adadelta' 'rmsprop'
              metrics=['accuracy'])

# the flow method yields batches of images indefinitely, with the given transofmrations
cnn.fit_generator(datagen.flow(X_subtrain, y_train_ohe, batch_size=128), 
                  steps_per_epoch=int(len(X_train)/128), # how many generators to go through per epoch
                  epochs=50, verbose=1,
                  validation_data=(X_sub_test,y_test_ohe),
                  callbacks=[EarlyStopping(monitor='val_loss', patience=2)]
                 )

ValueError: Error when checking input: expected conv2d_37_input to have shape (None, 32, 32, 1) but got array with shape (2954, 32, 32, 3)